# Anomaly Detection

In [1]:
#importing the libraries we'll need
import pandas as pd
import numpy as np

In [2]:
#importing the data
df = pd.read_csv(r'P:\ENGINEERING\FPO\Digitalisation\1 FPO Projects\IIX Data Analytics\02-General presentations\Knowledge folder - preparation\Data Science Learning Framework\Machine Learning Posters\6 - Anomaly Detection\sensor.csv')
#there are 51 sensors for the pump, and a label (0 for operational, 1 for broken) of the machine
df.head()

,machine_status,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50
0,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889
1,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889
2,0,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,...,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037
3,0,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,...,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250
4,0,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,...,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889


In [3]:
#separating into training, cross validation and test sets
df = df.dropna(axis=0, how="any") #dropping nulls
#training data, no positives as not needed and to save as many as possible for cross validation/ testing
x = np.array(df[df["machine_status"]==0].drop(["machine_status"], axis=1))
y = np.array(df[df["machine_status"]==0]["machine_status"])[:,np.newaxis]

x_pos = np.array(df[df["machine_status"]==1].drop(["machine_status"], axis=1)) #putting positive data into new array
x_pos = x_pos[np.random.permutation(len(x_pos))] #shuffling array
y_pos = np.array(df[df["machine_status"]==1]["machine_status"])[:,np.newaxis]

#creating random index to pull negative values from training set into cross validation and test sets
random_index = np.random.randint(len(x), size=(10000))            

#creating cross validation and test sets from positive and negative examples
x_cv = np.append(x[random_index[:5000],:], x_pos[:int(len(x_pos)/2)], axis=0)
y_cv = np.append(y[random_index[:5000],:], y_pos[int(len(y_pos)/2):], axis=0)
x_test = np.append(x[random_index[5000:],:], x_pos[:int(len(x_pos)/2)], axis=0)
y_test = np.append(y[random_index[5000:],:], y_pos[int(len(y_pos)/2):], axis=0)

#removing cross validation and test examples from training set
x = np.delete(x, random_index, axis=0)
y = np.delete(y, random_index, axis=0)

## Creating the Anomaly Detection Algorithm (as a class)

In [4]:
class anomaly_detection(object):
    def __init__(self, x):
        '''__init__ function to define the training data and their mean as objects within the class,
           as well as calculate the covariance matrix
           '''
        self.x = np.array(x)
        self.x_mean = np.mean(self.x, axis=0)
        def sigma_calc(self):
            '''calculating the covariance matrix
            '''
            return (1/len(self.x))*np.sum(np.array([np.dot(i[:,np.newaxis]-self.x_mean,
                                                          (i[:,np.newaxis]-self.x_mean).transpose())
                                                          for i in self.x]),axis=0)

        self.sigma = sigma_calc(self)

    def p_calc(self, x, x_mean):
        '''calculate p (Gaussian Distribution value) for all rows in the given dataset
           (can take a while as needs to calculate inverse of large matrix)
        '''
        return (1/((2*np.pi)**x.shape[1]*np.linalg.det(self.sigma)**0.5))*np.e**np.array([-0.5*np.dot( 
                                                                                                 np.dot(
                                                                                                       (i-x_mean),
                                                                                                       np.linalg.inv(self.sigma)), 
                                                                                                       ((i-x_mean).transpose())) 
                                                                                                       for i in x])

    def z_optimiser(self, p, y, x, iterations):
        '''optimises cut-off value of z based on maximising precision and recall, ideally using the cross validation set
        '''
        #creating vectors to fill with precision/recall values for different values of z
        pres_rec_vectors = np.zeros((iterations))
        precisions = np.zeros((iterations))
        recalls = np.zeros((iterations))
        #iterating over z
        for i, z in enumerate(np.linspace(min(p), max(p), iterations)):
            #define precision, recall and the euclidian length of them (to maximise for optimum z)
            precisions[i] = len(p[(p <= z) & (y == 1)])/(len(p[(p <= z) & (y == 1)])+len(p[(p <= z) & (y == 0)]))
            recalls[i] = len(p[(p <= z) & (y == 1)])/(len(p[(p <= z) & (y == 1)])+len(p[(p >= z) & (y == 1)]))
            pres_rec_vectors[i] = np.sqrt(precisions[i]**2+recalls[i]**2)
        
        #picking optimal z, and its precision and recall
        z_opt = np.linspace(min(p), max(p), iterations)[np.argmax(pres_rec_vectors)]
        pres_opt = precisions[np.argmax(pres_rec_vectors)]
        recall_opt = recalls[np.argmax(pres_rec_vectors)]

        print("z optimised at {}, with precision {} and recall {}".format(z_opt, np.round(pres_opt,3), np.round(recall_opt,3)))

        return z_opt

    def anom_pred_calc(self, p, z):
        '''returns a boolean vector fo whether p is bigger than z for all data rows, i.e. classifying as anomaly or not'''
        anom_pred = (p < z).astype(int)
        return anom_pred

In [5]:
ad = anomaly_detection(x) #creating anomaly detection object

## Performing Cross Validation to Optimise Z (using cross validation set)

In [6]:
p = ad.p_calc(x_cv, np.mean(x_cv, axis=0)) #calculating p for data in the cross validation set
z = ad.z_optimiser(p, y_cv.flatten(), x_cv, 10000) #optimising z

z optimised at 3.276566533284943e-143, with precision 0.83 and recall 0.996


## Assessing Performance on Test Set

In [7]:
p = ad.p_calc(x_test, np.mean(x_test, axis=0)) #calculating p for data in the test set
anom_pred = ad.anom_pred_calc(p, z) #predicting if anomaly or not (based on p & z)

In [8]:
summary_df = pd.DataFrame({"P":p, "label":y_test[:,0], "predicted_label":anom_pred}) #creating dataframe of output

In [9]:
print("Confusion matrix:")
print(len(summary_df[(summary_df["label"] == 1) & (summary_df["predicted_label"] == 1)]), len(summary_df[(summary_df["label"] == 0) & (summary_df["predicted_label"] == 1)]))
print(len(summary_df[(summary_df["label"] == 1) & (summary_df["predicted_label"] == 0)]), len(summary_df[(summary_df["label"] == 0) & (summary_df["predicted_label"] == 0)]))

#confusion matrix is of the form:
#top left number: True positives - no. of examples in the test set that were positive (pump down) and identified as such
#top right number: False positive - no. of examples predicted_label as postive but actually negative (pump operational)
#bottom left number: False negative - no. of examples predicted_label as negative but actually positive
#bottom right number: True negative - no. of examples predicted_label as negative and identified as such

Confusion matrix:
1476 314
9 4686


Our model has few false negatives or positives compared to correctly classified data, and as such we can be confident it will identify when the pump has failed much more accurately than without a model. The model has many more false positives than negatives, so we may be able to improve performance by increasing z a small amount, but it's important to do this based on the cross-validation set rather than the test set.